In [15]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import matplotlib.pyplot as plt
import numpy as np
from dppy.finite_dpps import FiniteDPP

from dataloader.builder import build_dataset
from uncertainty_estimator.mcdue import MCDUEMasked
from uncertainty_estimator.masks import build_masks
from model.mlp import MLP
from experiments.utils.data import scale

In [17]:
config = {
    'dataset': 'concrete',
    'layers': [8, 256, 256, 128, 1]
}


In [ ]:
def rank(dpp):
    N = dpp.eig_vecs.shape[0]
    tol = np.max(dpp.L_eig_vals) * N * np.finfo(np.float).eps
    rank = np.count_nonzero(dpp.L_eig_vals > tol)
    return rank

In [94]:
dataset = build_dataset(config['dataset'])
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')

x_train, x_val, _ = scale(x_train, x_val)
y_train, y_val, y_scaler = scale(y_train, y_val)


In [95]:
mask = build_masks(['adpp'])['adpp']

In [96]:
mask.reset()
model = MLP(config['layers'])
model.fit((x_train, y_train), (x_val, y_val), dropout_rate=0.2, epochs=500)

estimator = MCDUEMasked(model, nn_runs=100, dropout_mask=mask)
uncertainty = estimator.estimate(x_val)


Epoch [100/500], Loss: 0.1832, Validation loss: 0.1144
Epoch [200/500], Loss: 0.0940, Validation loss: 0.1437
Epoch [300/500], Loss: 0.0783, Validation loss: 0.0928
Epoch [400/500], Loss: 0.0903, Validation loss: 0.1388
Epoch [500/500], Loss: 0.0597, Validation loss: 0.0863


In [97]:
k_matrix = mask.layer_correlations[1]
dpp = FiniteDPP('likelihood', **{'L': np.abs(k_matrix)})
# dpp.sample_exact_k_dpp(10)
for i in range(50):
    dpp.sample_exact_k_dpp(10)
for sample in dpp.list_of_samples:
    print(sample)


[33  9 10 40 13 27 34 28 41 32]
[54 10 31 30  7 50  0 13 33 14]
[33  0 26 13 11 41 28 58 44 46]
[61 50  8 51 25 13 48 34 32 37]
[32 10 28  9 21 63  0  7 12 44]
[33 44  0 34 21 13 54  7 12  4]
[41 33  3 60 32  2 59 31 25  9]
[31  0  8 54 32 58 34 11 50 12]
[12  4 34  8 13 32 33 20 44  6]
[ 9 21 54 12 18 43 31 20  5  4]
[43 57 23 39  8 13 28  6 11  9]
[59 58 42 63 32 37  9 25 13 34]
[51 62  0 63 25 13 43  8  2 48]
[ 0 30 54 32 13 59  7 34 43 44]
[43 57 31 12  0 33 54 32 28  2]
[21 44 55  5  8 12 14 54 31  9]
[32 26 31  8 44 34 50 14  9 12]
[ 0 33 25 32 41 34 26 17 28 21]
[33 50 46 59 37 55 32 54  8  0]
[10 12 15  8 62 44 28 25 43 58]
[25 40  8 14 44 31 50  0  4 33]
[50 33 49 54  8 25 13 21 57 16]
[54 12 23 10  8 21  9 13 50 43]
[27  2  0  5 13 33 54 63 12 31]
[29 33 18  8 58 24 52 21 12  0]
[23 15 17 31 33 21  0 43 54 48]
[54  7 32 60 30 13 25 31 39 19]
[20 17  9 29 53 21 33 41 31 44]
[12 32 28 22 62 21 30  5 34 13]
[38 12 33  0 43 59  7 44 32 56]
[33 31 11 32 49 12 13  9 16 29]
[43 13  

In [23]:
# compare rans for the original and positive correlation matrices
# ranks = []
# ranks_abs = []
# for i in range(2):
#     print(i)
#     model = MLP(config['layers'])
#     model.fit((x_train, y_train), (x_val, y_val), verbose=False, epochs=2000)
#     mask = build_masks(['dpp'])['dpp']
#     estimator = MCDUEMasked(model, nn_runs=100, dropout_mask=mask)
#     uncertainty = estimator.estimate(x_val)
#     k_matrix = mask.layer_correlations[0]
#     dpp = FiniteDPP('correlation', **{'K': k_matrix})
#     dpp.sample_exact_k_dpp(1)
#     ranks.append(rank(dpp))
#     dpp = FiniteDPP('correlation', **{'K': np.abs(k_matrix)})
#     dpp.sample_exact_k_dpp(1)
#     ranks_abs.append(rank(dpp))

In [24]:
# plt.scatter(ranks, ranks_abs)
# plt.plot((0, max(ranks)), (0, max(ranks)))
# plt.xlabel('full range ranks')
# plt.ylabel('abs ranks')